In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import spacy
import seaborn as sns
import nltk
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
!python -m spacy download el_core_news_sm

In [ ]:
nlp = spacy.load('el_core_news_sm')

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), min_df=0.01, max_df=0.95)

In [ ]:
start_page = 1
end_page = 14

start_url = 'https://www.newsit.gr/page/'

extra_url = '/?s=%CE%B1%CE%BD%CF%84%CE%B9%CE%B5%CE%BC%CE%B2%CE%BF%CE%BB%CE%B9%CE%B1%CF%83%CF%84%CE%AD%CF%82'

test_url = start_url + str(start_page) + str(extra_url)

test_url

'https://www.newsit.gr/page/1/?s=%CE%B1%CE%BD%CF%84%CE%B9%CE%B5%CE%BC%CE%B2%CE%BF%CE%BB%CE%B9%CE%B1%CF%83%CF%84%CE%AD%CF%82'

In [ ]:
response = requests.get(test_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [ ]:
teaser_articles_list = doc.find('main', {'class': 'site-main'}).find_all('article')

In [ ]:
teaser_articles_list

In [ ]:
link = teaser_articles_list[0].find('a')['href']

link

'https://www.newsit.gr/politikh/ekloges-2023-niki-oxi-sta-kommata-me-antiepistimonikes-theseis-sti-xora-tou-ippokrati-leei-o-eksadaktylos/3785280/'

In [ ]:
teasers_list = []

for page_num in range(start_page, end_page+1):
  page_url = start_url + str(page_num) + str(extra_url)
  response = requests.get(page_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στo teaser_articles_list***
  teaser_articles_list = doc.find('main', {'class': 'site-main'}).find_all('article')


  for teaser in teaser_articles_list:
      story_dict = {}
      try:

        #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στο link, ΜΕΤΑ το teaser
        link = teaser.find('a')['href']


        if link.startswith("http"):
          story_dict['url'] = link
        else:
          story_dict['url'] = main_url + link
      except:
        story_dict['url'] = ''
        pass
      teasers_list.append(story_dict)
  time.sleep(1)

In [ ]:
newsit_teasers_df = pd.DataFrame(teasers_list)
newsit_teasers_df

,url
0,https://www.newsit.gr/politikh/ekloges-2023-ni...
1,https://www.newsit.gr/kosmos/koronoios-erxetai...
2,https://www.newsit.gr/kosmos/Unicef-67-ekat-pa...
3,https://www.newsit.gr/ygeia/karkinos-oi-synomo...
4,https://www.newsit.gr/kosmos/nea-zilandia-gone...
...,...
258,https://www.newsit.gr/politikh/mitsotakis-ypar...
259,https://www.newsit.gr/kosmos/eksallos-o-zeexof...
260,https://www.newsit.gr/kosmos/o-koronoios-einai...
261,https://www.newsit.gr/kosmos/koronoios-ypoxreo...


In [ ]:
newsit_teasers_df.to_csv('/content/temp/newsit_teasers.csv', index=False)

In [ ]:
article_url = 'https://www.newsit.gr/kosmos/gallia-diadiloseis-kata-tou-emvoliasmou-gia-ton-koronoio-se-polles-poleis/3327628/'

In [ ]:
response = requests.get(article_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [ ]:
article = doc.find('div', {'class': 'site-content'})

In [ ]:
section = doc.find('a', {'href': 'https://www.newsit.gr/category/kosmos/'}).text
section

'Κοσμος'

In [ ]:
title = article.find('h1', {'class': 'entry-title'}).text
title

'Γαλλία: Διαδηλώσεις κατά του εμβολιασμού για τον κορονοϊό σε πολλές πόλεις'

In [ ]:
date = article.find('time')['datetime']
date

'2021-07-17T21:25:18+03:00'

In [ ]:
paragraphs = article.find_all('p')
paragraphs

In [ ]:
p_texts_list = []                      # δημιουργία κενής λίστας για να αποθηκεύσουμε το text κάθε παραγράφου
paragraphs = article.find_all('p')     # λίστα με όλες τις παραγράφους
for p in paragraphs:                   # για κάθε μια από τις παραγράφους στη λίστα
  p_texts_list.append(p.text)          # πάρε το .text και πρόσθεσέ το στη λίστα με τα texts των παραγράφων
full_text = ' '.join(p_texts_list)     # συνένωση των texts των παραγράφων
full_text = "".join(full_text.splitlines())
full_text

In [ ]:
full_articles_list = []

for article_url in newsit_teasers_df['url']:
  response = requests.get(article_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στo article που βρήκατε παραπάνω***
  article = doc.find('div', {'class': 'site-content'})

  full_article_dict = {}

  # website
  full_article_dict['site'] = "newsit.gr"

  # url
  full_article_dict['url'] = article_url

  #section
  try:
    section = doc.find('a', {'href': 'https://www.newsit.gr/category/kosmos/'}).text
    full_article_dict['section'] = section
  except:
    try:
      section = doc.find('a', {'href': 'https://www.newsit.gr/category/ellada/'}).text
      full_article_dict['section'] = section
    except:
      full_article_dict['section'] = ''
      pass

  #title
  try:
    title = article.find('h1', {'class': 'entry-title'}).text
    full_article_dict['title'] = title
  except:
    full_article_dict['title'] = ''
    pass

  # date
  try:
    date = article.find('time')['datetime']
    full_article_dict['date'] = date
  except:
    full_article_dict['date'] = ''
    pass

  # author
  try:
    author = article.find('div', {'class', "article__author"}).span.text
    full_article_dict['author'] = author
  except:
    full_article_dict['author'] = ''
    pass

  # full_text
  try:
    p_texts_list = []
    paragraphs = article.find_all('p')
    for p in paragraphs:
      p_texts_list.append(p.text)
    full_text = ' '.join(p_texts_list)
    full_text = "".join(full_text.splitlines())
    full_article_dict['full_text'] = full_text
  except:
    full_article_dict['full_text'] = ''
    pass

  full_articles_list.append(full_article_dict)
  time.sleep(1)

In [ ]:
newsit_full_articles_df = pd.DataFrame(full_articles_list)
newsit_full_articles_df

,site,url,section,title,date,author,full_text
0,newsit.gr,https://www.newsit.gr/politikh/ekloges-2023-ni...,Κοσμος,Εκλογές 2023 – ΝΙΚΗ: Όχι στα κόμματα με αντιεπ...,2023-05-29T08:37:44+03:00,,NewsIT Πολιτικη Εξαδακτυλος για το «ΝΙΚΗ» πριν...
1,newsit.gr,https://www.newsit.gr/kosmos/koronoios-erxetai...,Κοσμος,Κορονοΐός: Έρχεται χιονοστιβάδα αγωγών κατά τω...,2023-05-02T19:19:28+03:00,,NewsIT Κοσμος Ερχεται χιονοστιβαδα αγωγων κατα...
2,newsit.gr,https://www.newsit.gr/kosmos/Unicef-67-ekat-pa...,Κοσμος,Unicef: 67 εκατ. παιδιά παγκοσμίως έχασαν δόσε...,2023-04-20T09:01:52+03:00,,NewsIT Κοσμος 67 εκατ. παιδια παγκοσμιως εχασα...
3,newsit.gr,https://www.newsit.gr/ygeia/karkinos-oi-synomo...,Κοσμος,Καρκίνος: Οι συνωμοσιολόγοι υιοθετούν μυθικές ...,2022-12-22T16:25:13+02:00,,NewsIT Υγεια Καρκινος: Οι συνωμοσιολογοι υιοθε...
4,newsit.gr,https://www.newsit.gr/kosmos/nea-zilandia-gone...,Κοσμος,Νέα Ζηλανδία: Γονείς αρνούνται να δοθεί «αίμα ...,2022-12-06T09:15:59+02:00,,NewsIT Κοσμος Γονεις αρνουνται να δοθει «αιμα ...
...,...,...,...,...,...,...,...
258,newsit.gr,https://www.newsit.gr/politikh/mitsotakis-ypar...,Κοσμος,Μητσοτάκης: Υπάρχει ήδη σχεδιασμός για το εμβό...,2020-11-28T14:08:54+02:00,,NewsIT Πολιτικη Μητσοτακης: Υπαρχει ηδη σχεδια...
259,newsit.gr,https://www.newsit.gr/kosmos/eksallos-o-zeexof...,Κοσμος,Έξαλλος ο Ζέεχοφερ με τη διαδήλωση κατά των μέ...,2020-08-30T13:28:00+03:00,,NewsIT Κοσμος Εξαλλος ο Ζεεχοφερ με τη διαδηλω...
260,newsit.gr,https://www.newsit.gr/kosmos/o-koronoios-einai...,Κοσμος,“Ο κορονοϊός είναι ένα μεγάλο ψέμα” – Διαδηλώσ...,2020-08-29T22:59:05+03:00,,NewsIT Κοσμος “Ο κορονοιος ειναι ενα μεγαλο ψε...
261,newsit.gr,https://www.newsit.gr/kosmos/koronoios-ypoxreo...,Κοσμος,Κορονοϊός: Υποχρεωτικός θα είναι ο εμβολιασμός...,2020-08-19T09:42:40+03:00,,NewsIT Κοσμος Κορονοιος: Υποχρεωτικος θα ειναι...


In [ ]:
newsit_full_articles_df['full_text'] = newsit_full_articles_df['full_text'].str.replace('πηγή:', ' ')

In [ ]:
newsit_full_articles_df

,site,url,section,title,date,author,full_text
0,newsit.gr,https://www.newsit.gr/politikh/ekloges-2023-ni...,Κοσμος,Εκλογές 2023 – ΝΙΚΗ: Όχι στα κόμματα με αντιεπ...,2023-05-29T08:37:44+03:00,,NewsIT Πολιτικη Εξαδακτυλος για το «ΝΙΚΗ» πριν...
1,newsit.gr,https://www.newsit.gr/kosmos/koronoios-erxetai...,Κοσμος,Κορονοΐός: Έρχεται χιονοστιβάδα αγωγών κατά τω...,2023-05-02T19:19:28+03:00,,NewsIT Κοσμος Ερχεται χιονοστιβαδα αγωγων κατα...
2,newsit.gr,https://www.newsit.gr/kosmos/Unicef-67-ekat-pa...,Κοσμος,Unicef: 67 εκατ. παιδιά παγκοσμίως έχασαν δόσε...,2023-04-20T09:01:52+03:00,,NewsIT Κοσμος 67 εκατ. παιδια παγκοσμιως εχασα...
3,newsit.gr,https://www.newsit.gr/ygeia/karkinos-oi-synomo...,Κοσμος,Καρκίνος: Οι συνωμοσιολόγοι υιοθετούν μυθικές ...,2022-12-22T16:25:13+02:00,,NewsIT Υγεια Καρκινος: Οι συνωμοσιολογοι υιοθε...
4,newsit.gr,https://www.newsit.gr/kosmos/nea-zilandia-gone...,Κοσμος,Νέα Ζηλανδία: Γονείς αρνούνται να δοθεί «αίμα ...,2022-12-06T09:15:59+02:00,,NewsIT Κοσμος Γονεις αρνουνται να δοθει «αιμα ...
...,...,...,...,...,...,...,...
258,newsit.gr,https://www.newsit.gr/politikh/mitsotakis-ypar...,Κοσμος,Μητσοτάκης: Υπάρχει ήδη σχεδιασμός για το εμβό...,2020-11-28T14:08:54+02:00,,NewsIT Πολιτικη Μητσοτακης: Υπαρχει ηδη σχεδια...
259,newsit.gr,https://www.newsit.gr/kosmos/eksallos-o-zeexof...,Κοσμος,Έξαλλος ο Ζέεχοφερ με τη διαδήλωση κατά των μέ...,2020-08-30T13:28:00+03:00,,NewsIT Κοσμος Εξαλλος ο Ζεεχοφερ με τη διαδηλω...
260,newsit.gr,https://www.newsit.gr/kosmos/o-koronoios-einai...,Κοσμος,“Ο κορονοϊός είναι ένα μεγάλο ψέμα” – Διαδηλώσ...,2020-08-29T22:59:05+03:00,,NewsIT Κοσμος “Ο κορονοιος ειναι ενα μεγαλο ψε...
261,newsit.gr,https://www.newsit.gr/kosmos/koronoios-ypoxreo...,Κοσμος,Κορονοϊός: Υποχρεωτικός θα είναι ο εμβολιασμός...,2020-08-19T09:42:40+03:00,,NewsIT Κοσμος Κορονοιος: Υποχρεωτικος θα ειναι...


In [ ]:
newsit_full_articles_df.to_csv('/content/temp/newsit_full_articles.csv', index=False)